Example usage of FlowClient to run daily location, modal location and flow queries.

In [2]:
import flowclient

## Create an access token
Generate an API access token using [FlowAuth](../../analyst/#flowauth).

## Create an API connection
Paste the access token from FlowAuth here as `TOKEN`

In [3]:
conn = flowclient.connect(url="http://localhost:9090", token=TOKEN)

## Daily location

### Specify query parameters

In [4]:
query_spec = flowclient.spatial_aggregate(
    locations=flowclient.daily_location(
        date="2016-01-01",
        aggregation_unit="admin3",
        method="last",
        subscriber_subset=None,
    )
)

query_spec

><div class="codehilite"><pre>{'locations': {'aggregation_unit': 'admin3',
               'date': '2016-01-01',
               'method': 'last',
               'query_kind': 'daily_location',
               'subscriber_subset': None},
 'query_kind': 'spatial_aggregate'}</pre></div>

### Set query running

In [5]:
query_id = flowclient.run_query(connection=conn, query=query_spec)
query_id

>`34061bc5e4bccbcf5f53bda5a2173939`

### Check status of query

In [6]:
flowclient.get_status(connection=conn, query_id=query_id)

>`Finished`

### Get query result

In [7]:
df = flowclient.get_result_by_query_id(connection=conn, query_id=query_id)
df.head()

pcod  total
0  524 1 01 01   1188
1  524 1 01 02     65
2  524 1 01 03    619
3  524 1 02 05   1170
4  524 1 02 06     25

Alternative (run query, wait for it to finish, and get result, in one function call):

In [8]:
df = flowclient.get_result(connection=conn, query=query_spec)
df.head()

pcod  total
0  524 1 01 01   1188
1  524 1 01 02     65
2  524 1 01 03    619
3  524 1 02 05   1170
4  524 1 02 06     25

## Modal location
### Specify query parameters
Either explicitly from a set of daily locations:

In [9]:
dl1 = flowclient.daily_location(
    date="2016-01-01", aggregation_unit="admin3", method="last", subscriber_subset=None
)
dl2 = flowclient.daily_location(
    date="2016-01-02", aggregation_unit="admin3", method="last", subscriber_subset=None
)
ml = flowclient.modal_location(locations=[dl1, dl2], aggregation_unit="admin3")

ml

><div class="codehilite"><pre>{'aggregation_unit': 'admin3',
 'locations': [{'aggregation_unit': 'admin3',
                'date': '2016-01-01',
                'method': 'last',
                'query_kind': 'daily_location',
                'subscriber_subset': None},
               {'aggregation_unit': 'admin3',
                'date': '2016-01-02',
                'method': 'last',
                'query_kind': 'daily_location',
                'subscriber_subset': None}],
 'query_kind': 'modal_location'}</pre></div>

Or from a date range:

In [10]:
ml = flowclient.modal_location_from_dates(
    start_date="2016-01-01",
    end_date="2016-01-03",
    aggregation_unit="admin3",
    method="last",
    subscriber_subset=None,
)

ml

><div class="codehilite"><pre>{'aggregation_unit': 'admin3',
 'locations': [{'aggregation_unit': 'admin3',
                'date': '2016-01-01',
                'method': 'last',
                'query_kind': 'daily_location',
                'subscriber_subset': None},
               {'aggregation_unit': 'admin3',
                'date': '2016-01-02',
                'method': 'last',
                'query_kind': 'daily_location',
                'subscriber_subset': None},
               {'aggregation_unit': 'admin3',
                'date': '2016-01-03',
                'method': 'last',
                'query_kind': 'daily_location',
                'subscriber_subset': None}],
 'query_kind': 'modal_location'}</pre></div>

### Set query running

In [11]:
query_id = flowclient.run_query(
    connection=conn, query=flowclient.spatial_aggregate(locations=ml)
)
query_id

>`017edf7186f5932f033f3abf45ef8b04`

### Check status of query

In [12]:
flowclient.get_status(connection=conn, query_id=query_id)

>`Finished`

### Get query result

In [13]:
df = flowclient.get_result_by_query_id(connection=conn, query_id=query_id)
df.head()

pcod  total
0  524 1 01 01   1398
1  524 1 01 02     39
2  524 1 01 03    732
3  524 1 02 05   1420
4  524 1 02 06     11

Alternative:

In [14]:
df = flowclient.get_result(
    connection=conn, query=flowclient.spatial_aggregate(locations=ml)
)
df.head()

pcod  total
0  524 1 01 01   1398
1  524 1 01 02     39
2  524 1 01 03    732
3  524 1 02 05   1420
4  524 1 02 06     11

## Flow
### Specify query parameters
Specify parameters for two location queries:

In [15]:
aggregation_unit = "admin1"
dl_past = flowclient.daily_location(
    date="2016-01-01",
    aggregation_unit=aggregation_unit,
    method="last",
    subscriber_subset=None,
)
dl_present = flowclient.daily_location(
    date="2016-01-07",
    aggregation_unit=aggregation_unit,
    method="last",
    subscriber_subset=None,
)

Specify parameters for flow query:

In [16]:
flow = flowclient.flows(
    from_location=dl_past, to_location=dl_present, aggregation_unit=aggregation_unit
)
flow

><div class="codehilite"><pre>{'aggregation_unit': 'admin1',
 'from_location': {'aggregation_unit': 'admin1',
                   'date': '2016-01-01',
                   'method': 'last',
                   'query_kind': 'daily_location',
                   'subscriber_subset': None},
 'query_kind': 'flows',
 'to_location': {'aggregation_unit': 'admin1',
                 'date': '2016-01-07',
                 'method': 'last',
                 'query_kind': 'daily_location',
                 'subscriber_subset': None}}</pre></div>

### Set query running

In [17]:
query_id = flowclient.run_query(connection=conn, query=flow)
query_id

>`ec1bc4f2e47dcc2cf441e766e532572b`

### Check status of query

In [18]:
flowclient.get_status(connection=conn, query_id=query_id)

>`Finished`

### Get query result

In [19]:
df = flowclient.get_result_by_query_id(connection=conn, query_id=query_id)
df.head()

count pcod_from pcod_to
0    557     524 1   524 5
1   1390     524 1   524 4
2    823     524 1   524 3
3    729     524 1   524 2
4   4246     524 1   524 1

## Counts of Events by Location

In [20]:
df = flowclient.get_result(
    connection=conn,
    query=flowclient.location_event_counts(
        start_date="2016-01-01",
        end_date="2016-01-08",
        aggregation_unit="admin3",
        count_interval="hour",
    ),
)
df.head()

date  hour         pcod  total
0  2016-01-01   0.0  524 1 01 01    112
1  2016-01-01   1.0  524 1 01 01    133
2  2016-01-01   2.0  524 1 01 01    136
3  2016-01-01   3.0  524 1 01 01    132
4  2016-01-01   4.0  524 1 01 01    114